In [1]:
import torch 
import pandas as pd
import torch.nn as nn
import time
import io
import joblib
import argparse

from torchsummary import summary
from tqdm import tqdm
from tabulate import tabulate
from sklearn.preprocessing import *

# from preprocessor import Preprocessor
# from AutoCleanse.utils import *
# from AutoCleanse.dataloader import PlainDataset, DataLoader
# from AutoCleanse.autoencoder import *
# from AutoCleanse.loss_model import loss_CEMSE
# # from AutoCleanse.preprocessor import Preprocessor
# from AutoCleanse.anonymize import anonymize
# from AutoCleanse.bucketfs_client import *

from utils import *
from dataloader import PlainDataset, DataLoader
from autoencoder import *
from masked_autoencoder import *
from loss_model import loss_CEMSE
from preprocessor import Preprocessor
from anonymize import anonymize
from bucketfs_client import *

c:\Users\zpakz\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device configuration

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

## Setup directory path

In [3]:
import os
PROJECT_DIR = os.getcwd()
os.chdir(PROJECT_DIR)
DATASET_DIR = os.path.join(PROJECT_DIR,'dataset')
EVAL_DIR = os.path.join(PROJECT_DIR,'evaluate')

## Preparing data

Load dataframe and group features by their type

In [4]:
df = pd.read_csv(os.path.join(DATASET_DIR,'adult.csv')).drop(columns=['fnlwgt','income'])
continous_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns.tolist()
og_columns = df.columns.to_list()
df = df[continous_columns+categorical_columns]

Data preprocessing

In [5]:
scaler = MinMaxScaler()
onehotencoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
preprocessor = Preprocessor(scaler,
                            onehotencoder,
                            continous_columns,
                            categorical_columns)

X_train,X_val,X_test = preprocessor.split(df=df,
                                        train_ratio=0.7,
                                        val_ratio=0.15,
                                        test_ratio=0.15,
                                        random_seed=42)
X_dirty = replace_with_nan(X_test,0.2,42)

X_train = preprocessor.fit_transform(input_df=X_train)
X_val = preprocessor.transform(input_df=X_val)        
X_test = preprocessor.transform(input_df=X_test)  

categories = preprocessor.encoder.categories_


In [6]:
batch_size = 64

train_dataset = PlainDataset(X_train)
val_dataset = PlainDataset(X_val)
test_dataset = PlainDataset(X_test)

def custom_collate_fn(batch):
    tensor_data = torch.stack([item[0] for item in batch])
    indices = [item[1] for item in batch]
    return tensor_data, indices

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True,collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

### Training MAE model and next the main AutoEncoder

In [7]:
layers = [X_train.shape[1],1024,128] 
wlc = (1,1) 

### MAE

In [8]:
# for the masked auto encoder, only training on the continuous section of the data. These columns will be used for inference in MAE,
# as this method is only used to replace the NAN values for these columns.

X_train_mae = X_train.iloc[:, 0:len(continous_columns)]
X_val_mae = X_val.iloc[:, 0:len(continous_columns)]
X_test_mae = X_test.iloc[:, 0:len(continous_columns)]

train_dataset_mae = PlainDataset(X_train_mae)
val_dataset_mae = PlainDataset(X_val_mae)
test_dataset_mae = PlainDataset(X_test_mae)

train_loader_mae = DataLoader(train_dataset_mae, batch_size=batch_size, shuffle=True, drop_last=True,collate_fn=custom_collate_fn)
val_loader_mae = DataLoader(val_dataset_mae, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
test_loader_mae = DataLoader(test_dataset_mae, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

In [9]:
## fit MAE model
layers_mae = [len(continous_columns),64,64,16] # layers for the MAE model, can be tuned

mae_autoencoder = MaskedAutoencoder(layers=layers_mae,dropout_enc=[(0,0.0)],dropout_dec=[(0,0.1)], batch_norm=True, \
                          learning_rate=1e-4,weight_decay=1e-5,l1_strength=1e-5,l2_strength=1e-5)


In [10]:
mae_autoencoder.train_model(
      patience=10,
      num_epochs=100,
      batch_size=batch_size,
      train_loader=train_loader_mae,
      val_loader=val_loader_mae,
      continous_columns=continous_columns, 
      categorical_columns=[], 
      categories=categories,
      device=device,
      wlc=wlc)

Epoch [1/100], Training Progress:   1%|▏         | 5/356 [00:00<00:11, 30.75it/s]

train progress: 
Epoch [1/100], Training Progress:   0%|          | 0/356 [00:00<?, ?it/s]


Epoch [1/100], Validation Progress: 100%|██████████| 76/76 [00:00<00:00, 143.32it/s]


Epoch [1/100], Training Loss: 0.09763432
Epoch [1/100], Validation Loss: 0.06671038
Epoch [1/100], Training CE Loss: 0.00000000
Epoch [1/100], Validation CE Loss: 0.00000000
Epoch [1/100], Training MSE Loss: 0.09763432
Epoch [1/100], Validation MSE Loss: 0.06671038
Epoch [1/100], Training Loss Comp: 0.09763432
Epoch [1/100], Validation Loss Comp: 0.06671038
Epoch [1/100]: Learning Rate = [0.0001]



Epoch [2/100], Training Progress:   3%|▎         | 11/356 [00:00<00:03, 103.97it/s]

...

Epoch [32/100], Training Loss: 0.00530529
Epoch [32/100], Validation Loss: 0.00556034
Epoch [32/100], Training CE Loss: 0.00000000
Epoch [32/100], Validation CE Loss: 0.00000000
Epoch [32/100], Training MSE Loss: 0.00530529
Epoch [32/100], Validation MSE Loss: 0.00556034
Epoch [32/100], Training Loss Comp: 0.00530529
Epoch [32/100], Validation Loss Comp: 0.00556034
Epoch [32/100]: Learning Rate = [1e-05]



Epoch [33/100], Training Progress:   6%|▌         | 21/356 [00:00<00:03, 103.28it/s]

train progress: 
Epoch [33/100], Training Progress:   0%|          | 0/356 [00:00<?, ?it/s]


Epoch [33/100], Validation Progress: 100%|██████████| 76/76 [00:00<00:00, 125.79it/s]

Epoch [33/100], Training Loss: 0.00535268
Epoch [33/100], Validation Loss: 0.00570360
Epoch [33/100], Training CE Loss: 0.00000000
Epoch [33/100], Validation CE Loss: 0.00000000
Epoch [33/100], Training MSE Loss: 0.00535268
Epoch [33/100], Validation MSE Loss: 0.00570360
Epoch [33/100], Training Loss Comp: 0.00535268
Epoch [33/100], Validation Loss Comp: 0.00570360
Epoch [33/100]: Learning Rate = [1e-05]

Early stopping triggered. Stopping training.


In [11]:
# mae_autoencoder.save("local","MAE")

In [12]:
# mae_autoencoder.load("local","MAE")

In [13]:
# pass in the attributes: "mean", "median", or "distribution_based" to the 'method' attribute to fill in the nan values using these methods, 
# or leave the 'methods' attribute empty and pass in the list to the 'MAE_pack' to fill the values using the Masked Auto Encoder method 
X_dirty = preprocessor.transform(input_df=X_dirty,  method="MAE", MAE_pack=[mae_autoencoder, batch_size, device, custom_collate_fn, test_loader_mae]) 

Clean progress:   0%|          | 0/76 [00:00<?, ?it/s]

Clean progress: 100%|██████████| 76/76 [00:00<00:00, 81.66it/s]


MAE: 0.04055763

MSE: 0.00629703


In [18]:
X_dirty

,age,education.num,capital.gain,capital.loss,hours.per.week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native.country_Mexico,native.country_Nicaragua,native.country_Philippines,native.country_Poland,native.country_Puerto-Rico,native.country_South,native.country_Taiwan,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam
847,0.103485,0.534719,0.187809,0.114989,0.471664,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
739,-0.161607,0.369771,0.218976,0.170122,0.413305,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,0.071218,0.352251,0.128771,0.063953,0.375419,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
704,0.019593,0.064369,0.375381,0.069501,0.394139,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,-0.007432,0.231049,0.437667,-0.016125,0.366932,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.019460,-0.001403,0.331468,0.114764,0.416940,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,0.036557,0.487635,0.086724,0.169354,0.449624,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
584,0.019460,-0.001403,0.331468,0.114764,0.416940,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
680,0.067823,0.518641,0.126017,0.142384,0.448606,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


(Optional) Check save/load function of preprocessor

Both functiona take in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **preprocessor_main.pkl**
    - Save/load location: can either be "local" to save/load in the home folder or "bucketfs" to save/load to/from Exasol BucketFS

In [23]:
preprocessor.save("main","local")
# preprocessor.save("main","bucketfs")
# preprocessor = Preprocessor(scaler=MinMaxScaler(),encoder=OneHotEncoder(sparse=False))
# preprocessor2.load("main","local")
# preprocessor2.load("main","bucketfs")

Convert dataframes into datasets, and create dataloaders

In [14]:
dirty_dataset = PlainDataset(X_dirty)

dirty_loader = DataLoader(dirty_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

## Instantiate model

In [15]:
autoencoder = Autoencoder(layers=layers,dropout_enc=[(0,0.0)],dropout_dec=[(0,0.1)], batch_norm=True, \
                          learning_rate=1e-4,weight_decay=1e-5,l1_strength=1e-5,l2_strength=1e-5)

In [27]:
summary(autoencoder.to(device),torch.tensor(X_train.values).float().to(device).shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]          95,232
       BatchNorm1d-2                 [-1, 1024]           2,048
              ReLU-3                 [-1, 1024]               0
           Dropout-4                 [-1, 1024]               0
            Linear-5                  [-1, 128]         131,200
       BatchNorm1d-6                  [-1, 128]             256
              ReLU-7                  [-1, 128]               0
            Linear-8                 [-1, 1024]         132,096
              ReLU-9                 [-1, 1024]               0
          Dropout-10                 [-1, 1024]               0
           Linear-11                   [-1, 92]          94,300
             ReLU-12                   [-1, 92]               0
           Linear-13                   [-1, 92]           8,556
Total params: 463,688
Trainable params:

(Optional) Model can be loaded from checkpoint after instantiation

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

### Train the model

In [16]:
autoencoder.train_model(
      patience=10,
      num_epochs=1,
      batch_size=batch_size,
      train_loader=train_loader,
      val_loader=val_loader,
      continous_columns=continous_columns, 
      categorical_columns=categorical_columns, 
      categories=categories,
      device=device,
      wlc=wlc)

Epoch [1/1], Validation Progress: 100%|██████████| 76/76 [00:02<00:00, 25.57it/s]

Epoch [1/1], Training Loss: 6.09625684
Epoch [1/1], Validation Loss: 2.12896782
Epoch [1/1], Training CE Loss: 6.05852925
Epoch [1/1], Validation CE Loss: 2.10090744
Epoch [1/1], Training MSE Loss: 0.03772758
Epoch [1/1], Validation MSE Loss: 0.02806038
Epoch [1/1], Training Loss Comp: 6.09625684
Epoch [1/1], Validation Loss Comp: 2.12896782
Epoch [1/1]: Learning Rate = [0.0001]



(Optional) Model can be saved after training

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [29]:
# autoencoder.save("local","main")
# autoencoder.load("local","main")
autoencoder.save("bucketfs","main",url="http://172.18.0.2:6583",bucket="default",user="w",password="write")

### Use trained model to clean data

In [17]:
cleaned_data = autoencoder.clean(
    dirty_loader=dirty_loader,
    test_loader=test_loader,
    df=X_dirty,
    batch_size=batch_size,
    continous_columns=continous_columns, 
    categorical_columns=categorical_columns, 
    og_columns=og_columns,
    onehotencoder=preprocessor.encoder, 
    scaler=preprocessor.scaler,
    device=device) 

Clean progress: 100%|██████████| 76/76 [00:06<00:00, 12.58it/s]


MAE: 0.02430817

MSE: 0.01999156


In [ ]:
cleaned_data = autoencoder.clean(
    dirty_loader=dirty_loader,
    test_loader=test_loader,
    df=X_dirty,
    batch_size=batch_size,
    continous_columns=continous_columns, 
    categorical_columns=categorical_columns, 
    og_columns=og_columns,
    onehotencoder=preprocessor.encoder, 
    scaler=preprocessor.scaler,
    device=device) 

Clean progress: 100%|██████████| 76/76 [00:06<00:00, 12.58it/s]


MAE: 0.02430817

MSE: 0.01999156


In [ ]:
# original data
print(tabulate(df.loc[[28296,28217,8054,4223,22723],og_columns],headers=og_columns,tablefmt="simple",maxcolwidths=[None, 4]))

In [ ]:
# cleaned data
print(tabulate(cleaned_data.loc[[28296,28217,8054,4223,22723]],headers=cleaned_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 4]))

### Use trained model to anonymize data

In [18]:
anonymized_data = autoencoder.anonymize(df=X_test,
                                        data_loader=test_loader,
                                        batch_size=batch_size,
                                        device=device)

Anonymize progress: 100%|██████████| 76/76 [00:02<00:00, 26.65it/s]


In [ ]:
# anonymized data
print(tabulate(anonymized_data.round(decimals=4).iloc[:5,:32],headers=anonymized_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 6]))